In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns

import lasagne
import theano
import theano.tensor as T

%matplotlib inline

In [3]:
train_dat = pd.read_csv('train.csv')
test_dat = pd.read_csv('test.csv')
# Target encoding
lbl = preprocessing.LabelEncoder()
lbl.fit(list(train_dat.target.values))
train_dat.target = lbl.transform(list(train_dat.target.values))
# Extract labels
train_labels = train_dat.target
# Drop labels
train_dat = train_dat.drop('target',axis=1)
# Drop id columns
train_dat = train_dat.drop('id',axis=1)
test_dat = test_dat.drop('id',axis=1)

In [19]:
n_classes = train_labels.nunique()
n_features = len(train_dat.columns)

In [37]:
l_in = lasagne.layers.InputLayer((None, n_features))
l_in_drop = lasagne.layers.DropoutLayer(l_in, p=0.1)
l_hidden = lasagne.layers.DenseLayer(l_in_drop, num_units=400, nonlinearity=lasagne.nonlinearities.sigmoid)
l_out = lasagne.layers.DenseLayer(l_hidden, num_units=n_classes,nonlinearity=T.nnet.softmax)

net_output = lasagne.layers.get_output(l_out)

true_output = T.ivector('true_output')
loss = T.mean(lasagne.objectives.categorical_crossentropy(net_output, true_output))

In [38]:
all_params = lasagne.layers.get_all_params(l_out)
updates = lasagne.updates.sgd(loss, all_params, learning_rate=1)

train = theano.function([l_in.input_var, true_output], loss, updates=updates)
get_output = theano.function([l_in.input_var], net_output)

In [25]:
N = int( len(test_dat)*0.8 )
print(train_dat.shape)
for i in range(2):
    train(train_dat[:N].as_matrix(), train_labels[:N])

(61878, 93)


In [28]:
y_predicted = np.argmax(get_output(train_dat[N:]), axis=1)
print(y_predicted)
print(np.mean(y_predicted[N:] == train_labels[N:]))

[]
nan


In [41]:
BATCH_SIZE = 61000
N_EPOCHS = 100
# Keep track of which batch we're training with
batch_idx = 0
# Keep track of which epoch we're on
epoch = 0
while epoch < N_EPOCHS:
    # Extract the training data/label batch and update the parameters with it
    train(train_dat.iloc[batch_idx:batch_idx + BATCH_SIZE],
          train_labels[batch_idx:batch_idx + BATCH_SIZE])
    batch_idx += BATCH_SIZE
    # Once we've trained on the entire training set...
    if batch_idx >= N:
        # Reset the batch index
        batch_idx = 0
        # Update the number of epochs trained
        epoch += 1
        # Compute the network's output on the validation data
        val_output = get_output(train_dat.iloc[:N])
        # The predicted class is just the index of the largest probability in the output
        val_predictions = np.argmax(val_output, axis=1)
        # The accuracy is the average number of correct predictions
        accuracy = np.mean(val_predictions == train_labels[:N])
        print("Epoch {} validation accuracy: {}".format(epoch, accuracy))

Epoch 1 validation accuracy: 0.6734218946960148
Epoch 2 validation accuracy: 0.6702543715052199
Epoch 3 validation accuracy: 0.6721613497527392
Epoch 4 validation accuracy: 0.6709977698051004
Epoch 5 validation accuracy: 0.671110895633343
Epoch 6 validation accuracy: 0.67195125892886
Epoch 7 validation accuracy: 0.6740521671676525
Epoch 8 validation accuracy: 0.6741491321632891
Epoch 9 validation accuracy: 0.6727108180613465
Epoch 10 validation accuracy: 0.6731633213743172
Epoch 11 validation accuracy: 0.6749248521283816
Epoch 12 validation accuracy: 0.6752965512783219
Epoch 13 validation accuracy: 0.6774782636801449
Epoch 14 validation accuracy: 0.6762985228998998
Epoch 15 validation accuracy: 0.680047836064514
Epoch 16 validation accuracy: 0.677849962830085
Epoch 17 validation accuracy: 0.6803387310514237
Epoch 18 validation accuracy: 0.6792074727689971
Epoch 19 validation accuracy: 0.6799831927340897
Epoch 20 validation accuracy: 0.6815992759946993
Epoch 21 validation accuracy: 0.68

KeyboardInterrupt: 

In [43]:
#xg_pred = xgb.DMatrix( test_dat )

# predict on test set
preds = get_output(test_dat)

samp = pd.read_csv('sampleSubmission.csv')
preds = pd.DataFrame(preds, index=samp.id.values, columns=samp.columns[1:])
preds.to_csv('submissionLasagne.csv', index_label='id')